In [47]:
import pandas as pd
import tensorflow as tf
import tensorflow_recommenders as tfrs
from tensorflow.keras import layers

# Sample interaction data
data = {
    'user_id': ['U1', 'U2', 'U3', 'U1', 'U2', 'U3'],
    'video_id': ['V1', 'V2', 'V3', 'V4', 'V1', 'V5'],
    'liked': [1, 0, 1, 1, 0, 1]  # Binary interaction as a list
}

# Create a DataFrame
interaction_df = pd.DataFrame(data)

# Prepare unique user IDs and video IDs
user_ids = interaction_df['user_id'].unique()
video_ids = interaction_df['video_id'].unique()

# Create lookup layers for user IDs and video IDs
user_lookup = layers.StringLookup(vocabulary=user_ids, mask_token=None)
video_lookup = layers.StringLookup(vocabulary=video_ids, mask_token=None)

In [49]:
# Define the recommendation model
class VideoRecommendationModel(tfrs.Model):
    def __init__(self):
        super().__init__()
        # Defining the user and video embeddings
        self.user_embedding = layers.Embedding(user_lookup.vocabulary_size(), 32)
        self.video_embedding = layers.Embedding(video_lookup.vocabulary_size(), 32)

    # Implementing the compute_loss method
    def compute_loss(self, features, training=False):
        # Lookup user and video IDs
        user_ids = user_lookup(features['user_id'])
        video_ids = video_lookup(features['video_id'])

        # Get the embeddings for user and video IDs
        user_embedding = self.user_embedding(user_ids)
        video_embedding = self.video_embedding(video_ids)
        
        # Calculate the binary cross-entropy loss
        return tf.reduce_mean(tf.keras.losses.binary_crossentropy(video_embedding, user_embedding))

In [50]:
# Convert interaction data to TensorFlow dataset
train_data = tf.data.Dataset.from_tensor_slices({
    'user_id': interaction_df['user_id'].astype(str).values,
    'video_id': interaction_df['video_id'].astype(str).values,
}).batch(2)  # Batch size can be adjusted as needed

# Instantiate the model
model = VideoRecommendationModel()

# Compile the model with an optimizer
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

# Train the model
model.fit(train_data, epochs=5)


Epoch 1/5
3/3 [==============================] - 1s 4ms/step - loss: -0.1140 - regularization_loss: 0.0000e+00 - total_loss: -0.1140
Epoch 2/5
3/3 [==============================] - 0s 13ms/step - loss: -0.9194 - regularization_loss: 0.0000e+00 - total_loss: -0.9194
Epoch 3/5
3/3 [==============================] - 0s 10ms/step - loss: -1.8158 - regularization_loss: 0.0000e+00 - total_loss: -1.8158
Epoch 4/5
3/3 [==============================] - 0s 9ms/step - loss: -2.7558 - regularization_loss: 0.0000e+00 - total_loss: -2.7558
Epoch 5/5
3/3 [==============================] - 0s 18ms/step - loss: -3.5796 - regularization_loss: 0.0000e+00 - total_loss: -3.5796


In [51]:
# Function to recommend videos for a given user ID
def recommend_videos(user_id, num_recommendations=3):
    # Ensure the user_id is in string format and convert to index
    user_vector = user_lookup(tf.convert_to_tensor([user_id], dtype=tf.string))
    
    # Get video IDs as tensor and their embeddings
    video_ids_tensor = video_lookup(tf.convert_to_tensor(video_ids, dtype=tf.string))
    video_vectors = model.video_embedding(video_ids_tensor)

    # Manage shape for user_vector
    user_vector = tf.expand_dims(user_vector, axis=0)  # Shape (1,)

    # Calculate scores between user and video embeddings
    scores = tf.matmul(model.user_embedding(user_vector), video_vectors, transpose_b=True)

    # Get the indices of the top recommendations
    video_indices = tf.argsort(scores, axis=1, direction='DESCENDING')

    return video_ids[video_indices.numpy()[0][:num_recommendations]]


In [52]:
# Example usage of the recommend_videos function
recommended_videos = recommend_videos(user_id='U1')
print("Recommended Videos for User U1:", recommended_videos)

Recommended Videos for User U1: [['V1' 'V1' 'V1' 'V1' 'V1']]
